In [1]:
import yfinance as yf
import datetime
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from scipy import stats

In [2]:
# YEARS MONTH DAY
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=2*365)

In [3]:
start_date

datetime.datetime(2022, 6, 11, 13, 30, 59, 962278)

In [4]:
def StockCheck(stock1,stock2):
    try:
        # print(stock1,stock2)
        data = yf.download(stock1+" "+stock2, start=start_date, end=end_date)
        # Take closing Value only
        data = data["Close"]

        data.fillna(method = 'ffill', inplace= True)
        data.fillna(method = 'bfill', inplace= True)


        # Ratio Calculation
        data['ratio'] = data[stock1]/data[stock2]
        # Corelation Caculation
        cor = data[stock1].corr(data[stock2])
        if cor < 0.88:
            return None, None, None, None, None, None, None, None, None, None
        
        xdata = data[stock1]
        ydata = data[stock2]
        latestx = xdata.iloc[-1]
        latesty = ydata.iloc[-1]

        # Calculate Intercept Standard Error
        result_xy = stats.linregress(xdata, ydata)
        stder_xy = result_xy.intercept_stderr
        result_yx = stats.linregress(ydata, xdata)
        stder_yx = result_xy.intercept_stderr


        # First part
        # We use Ordinary Least Squares method to find the line of best fit
        model_xy = sm.OLS(ydata, sm.add_constant(xdata)).fit()
        influence_xy = model_xy.get_influence()
        standardized_residuals_xy = influence_xy.resid
        residualerror_xy = standardized_residuals_xy.std()
        stdres_xy = influence_xy.resid_studentized_internal
        rat_xy = stder_xy/residualerror_xy
                
        #Outputs

        # Part 2
        model_yx = sm.OLS(xdata, sm.add_constant(ydata)).fit()
        influence_yx = model_yx.get_influence()
        standardized_residuals_yx = influence_yx.resid
        residualerror_yx = standardized_residuals_yx.std()
        stdres_yx = influence_yx.resid_studentized_internal
        rat_yx = stder_yx/residualerror_yx

        # Outputs
        # if first regression is smaller
        if rat_xy < rat_yx:
            adfTest = adfuller(standardized_residuals_xy, autolag='AIC')
            beta = result_xy.slope
            intercept = result_xy.intercept
            std_residual = residualerror_xy
            lastest_stdresidual = stdres_xy[-1]
            lastest_residual = standardized_residuals_xy[-1]
            ratio = (1-(abs(intercept)/latesty))*100
            return (beta, intercept, std_residual, lastest_stdresidual, lastest_residual, cor, adfTest[1], ratio, stock1, stock2)
        else:
            adfTest = adfuller(standardized_residuals_yx, autolag='AIC')
            beta = result_yx.slope
            intercept = result_yx.intercept
            std_residual = residualerror_yx
            lastest_stdresidual = stdres_yx[-1]
            lastest_residual = standardized_residuals_yx[-1]
            ratio = (1-(abs(intercept)/latestx))*100
            return (beta, intercept, std_residual, lastest_stdresidual, lastest_residual, cor, adfTest[1], ratio, stock2, stock1)
            
    except Exception as e:
        print(e)
        return(0,0,0,0,0,0,0,0,0,0)

In [6]:
# company1=[]
# company2=[]
# Sector = []
# P_values=[]
# beta_values=[]
# intercept_values=[]
# std_residual_values=[]
# lastest_stdresidual_values=[]
# lastest_residual_values=[]
# cor_values=[]
# ratios=[]


In [7]:
stime = datetime.datetime.now()

In [8]:
stime

datetime.datetime(2024, 6, 10, 13, 31, 9, 114031)

In [9]:
# data = pd.read_csv("Input/FNO_LIST.csv")
data = pd.read_csv(r"C:\Users\acer\Desktop\Py-Fin\Fin_data\Fin_data\Input\FNO_LIST.csv")
from itertools import combinations 

In [10]:
data

,SYMBOLS,Sector,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,ADANIENT.NS,1,ADANIENT,300.0,65.5,NaN,NaN,NaN,NaN
1,INDIGO.NS,11,INDIGO,300.0,18.5,NaN,NaN,NaN,NaN
2,MOTHERSON.NS,111,MOTHERSON,7100.0,20.5,NaN,NaN,NaN,NaN
3,SBICARD.NS,1111,SBICARD,800.0,18.5,NaN,NaN,NaN,NaN
4,SHRIRAMFIN.NS,11111,SHRIRAMFIN,300.0,21.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
270,IDFC.NS,IDFC GROUP,IDFC,5000.0,20.5,IDFC GROUP,NaN,NaN,NaN
271,IDFCFIRSTB.NS,IDFC GROUP,IDFCFIRSTB,7500.0,22.5,IDFC GROUP,1.0,NaN,NaN
272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,M&M.NS,M&M GROUP,M&M,350.0,17.5,M&M GROUP,NaN,NaN,NaN


In [11]:
# group all the data based on sector and store it in grouped data
grouped = data.groupby('Sector')
resultList = []

In [12]:
#key = Sector and group = Data of that sector
# key is Example IT
# group is the symbol of that key
for key, group in grouped:

    # Make it optimised in making pairs

    for b, g in combinations(group['SYMBOLS'].tolist(), 2):
        resultList.append([b, g, key])
        
resultList.sort()

In [13]:
resultList

[['AARTIIND.NS', 'DEEPAKNTR.NS', 'Specialty Chemicals'],
 ['AARTIIND.NS', 'NAVINFLUOR.NS', 'Specialty Chemicals'],
 ['AARTIIND.NS', 'SRF.NS', 'Specialty Chemicals'],
 ['ABB.NS', 'BHEL.NS', 'Electric and Power Equipment'],
 ['ABB.NS', 'SIEMENS.NS', 'Electric and Power Equipment'],
 ['ABBOTINDIA.NS', 'ALKEM.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'AUROPHARMA.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'BIOCON.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'CIPLA.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'DIVISLAB.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'DRREDDY.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'GLENMARK.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'GRANULES.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'IPCALAB.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'LAURUSLABS.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'LUPIN.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'SUNPHARMA.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'SYNGENE.NS', 'Pharmaceuticals'],
 ['ABBOTINDIA.NS', 'TORNTP

In [14]:
rl=[resultList[0],resultList[1],resultList[4]]
rl

[['AARTIIND.NS', 'DEEPAKNTR.NS', 'Specialty Chemicals'],
 ['AARTIIND.NS', 'NAVINFLUOR.NS', 'Specialty Chemicals'],
 ['ABB.NS', 'SIEMENS.NS', 'Electric and Power Equipment']]

In [15]:
# for i in list(rl)[0:] :
    
#     print(i)
    
# def lboom(i):
#     beta,intercept,std_residual,lastest_stdresidual,lastest_residual,cor,adfTest, ratio, x_axis, y_axis = StockCheck(i[0],i[1])    
#     #print(cor)
#     if cor == None:
#         # continue
#         return
        
#     if adfTest<0.10 and cor>0.88:     
#         company1.append(x_axis)
#         company2.append(y_axis)
#         Sector.append(i[2])
#         beta_values.append(beta)
#         intercept_values.append(intercept)
#         std_residual_values.append(std_residual)
#         lastest_stdresidual_values.append(lastest_stdresidual)
#         lastest_residual_values.append(lastest_residual)
#         cor_values.append(cor)
#         P_values.append(adfTest)
#         ratios.append(ratio)

In [16]:
# for i2 in rl:
#     lboom(i2)

In [17]:
# for i in list(rl)[0:] :
    
#     print(i)
    
#     beta,intercept,std_residual,lastest_stdresidual,lastest_residual,cor,adfTest, ratio, x_axis, y_axis = StockCheck(i[0],i[1])    
#     #print(cor)
#     print(adfTest,cor)
#     if cor == None:
#         continue
#     if adfTest<0.10 and cor>0.88:     
#         company1.append(x_axis)
#         company2.append(y_axis)
#         Sector.append(i[2])
#         beta_values.append(beta)
#         intercept_values.append(intercept)
#         std_residual_values.append(std_residual)
#         lastest_stdresidual_values.append(lastest_stdresidual)
#         lastest_residual_values.append(lastest_residual)
#         cor_values.append(cor)
#         P_values.append(adfTest)
#         ratios.append(ratio)

In [18]:
# pd1 = pd.DataFrame()
# pd1['Company2 (Y)'] = company2
# pd1['Company1 (X)'] = company1
# pd1['Sector'] = Sector
# pd1['P-value']=P_values
# pd1['beta']=beta_values
# pd1['intercept']=intercept_values
# pd1['std_residual']=std_residual_values
# pd1['Normalised_residual']=lastest_stdresidual_values
# pd1['Error for that day']=lastest_residual_values
# pd1['Correlation'] = cor_values
# pd1['ratio'] = ratios

In [19]:
# # pd1.to_csv('Output/Stocks.csv')
# pd1.to_csv(r'C:\Users\acer\Desktop\Py-Fin\Fin_data\Fin_data\Output\New_Stocks.csv')
# etime = datetime.datetime.now()
# print(f"{etime-stime} time taken to run script")

In [ ]:
import threading
import time

def sleep_function(seconds):
  """
  This function simulates a task that takes a specified number of seconds to complete.
  """
  print(f"Simulating work for {seconds} seconds...")
  time.sleep(seconds)
  print(f"Work completed after {seconds} seconds!")

# Define a list of sleep durations
sleep_durations = [2, 4, 1]  # List containing seconds to sleep for each thread

# Create threads and start them
threads = []
for duration in sleep_durations:
  thread = threading.Thread(target=sleep_function, args=(duration,))
  threads.append(thread)
  thread.start()

# Wait for all threads to finish
for thread in threads:
  thread.join()

print("All threads completed!")


In [ ]:
import threading
import queue
import time

def sleep_function(seconds, results_queue):
  """
  This function simulates a task and adds the sleep duration and its doubled value 
  to a queue.
  """
  print(f"Simulating work for {seconds} seconds...")
  time.sleep(seconds)
  results_queue.put([seconds, seconds * 2])  # Add result to queue

# Define a list of sleep durations
sleep_durations = [2, 4, 1]

# Create a queue to store results
results_queue = queue.Queue()

# Create threads
threads = []
for duration in sleep_durations:
  thread = threading.Thread(target=sleep_function, args=(duration, results_queue))
  threads.append(thread)
  thread.start()

# Wait for all threads to finish
for thread in threads:
  thread.join()

# Get results from the queue
results = []
while not results_queue.empty():
  result = results_queue.get()
  results.append(result)

# Create DataFrame from results
import pandas as pd
df = pd.DataFrame(results, columns=["sleep_duration", "doubled_duration"])

print(df)


In [ ]:
sleep_function(2)
sleep_function(4)
sleep_function(1)

In [ ]:
rl

In [26]:
import threading
import queue


def lboom(i,results_queue):
    beta, intercept, std_residual, lastest_stdresidual, lastest_residual, cor, adfTest, ratio, x_axis, y_axis = StockCheck(i[0], i[1])
    row=[]
    print('Func called')
    if cor is None:
        print('Nai mila')
        return

    if adfTest < 0.10 and cor > 0.88:
        print("mil gaya")
        row.append(x_axis)
        row.append(y_axis)
        row.append(i[2])
        row.append(beta)
        row.append(intercept)
        row.append(std_residual)
        row.append(lastest_stdresidual)
        row.append(lastest_residual)
        row.append(cor)
        row.append(adfTest)
        row.append(ratio)
        print(row)
        # results_queue.put(row)

In [29]:
rl

[['AARTIIND.NS', 'DEEPAKNTR.NS', 'Specialty Chemicals'],
 ['AARTIIND.NS', 'NAVINFLUOR.NS', 'Specialty Chemicals'],
 ['ABB.NS', 'SIEMENS.NS', 'Electric and Power Equipment']]

In [28]:
results_queue = queue.Queue()

threads = []
for pair in rl:
    thread=threading.Thread(target=lboom,args=(pair,results_queue))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()
    
    
# results = []
# while not results_queue.empty():
#     result = results_queue.get()
#     results.append(result)

# # Create DataFrame from results
# import pandas as pd
# df = pd.DataFrame(results, columns=["1","1","1","1","1","1","1","1","1","1",'1' ])

# df

[*********************100%%**********************]  2 of 2 completed

'ABB.NS'Func called
Nai mila
'NAVINFLUOR.NS'
Func called

Func called





[*********************************150%%**********************************]  3 of 2 completed

In [23]:
lboom(rl[2],0)

[*********************100%%**********************]  2 of 2 completed

Func called
mil gaya
['SIEMENS.NS', 'ABB.NS', 'Electric and Power Equipment', 1.288745124200773, -660.1131144433716, 250.1615258297903, -0.5118401568471248, -126.88861480337619, 0.9824598377808326, 0.005640541922076871, 91.82345258175614]


In [20]:
rl

[['AARTIIND.NS', 'DEEPAKNTR.NS', 'Specialty Chemicals'],
 ['AARTIIND.NS', 'NAVINFLUOR.NS', 'Specialty Chemicals'],
 ['ABB.NS', 'SIEMENS.NS', 'Electric and Power Equipment']]

In [ ]:
# results_queue = queue.Queue()


# def worker(data):
#     threads = []
#     for i in data:
#         # Create a thread object for each inner list
#         t = threading.Thread(target=lboom, args=(i,))
#         threads.append(t)
#         # Start the thread
#         t.start()
#     # Wait for all threads to finish
#     for thread in threads:
#         thread.join()


# data = [
#     ['ABBOTINDIA.NS', 'DRREDDY.NS', 'Pharmaceuticals'],
#     ['AARTIIND.NS', 'NAVINFLUOR.NS', 'Specialty Chemicals'],
#     ['AARTIIND.NS', 'SRF.NS', 'Specialty Chemicals']
# ]

# worker(data)

# print(company1)
# print(company2)
# print(Sector)
# print(beta_values)
# print(intercept_values)
# print(std_residual_values)
# print(lastest_stdresidual_values)
# print(lastest_residual_values)
# print(cor_values)
# print(P_values)
# print(ratios)
